In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

from tqdm import tqdm

from PIL import Image
import requests

import warnings
warnings.filterwarnings('ignore')

In [55]:
data_path = "/opt/ml/data/"

In [56]:
users = pd.read_csv(data_path + 'users.csv')
books = pd.read_csv(data_path + 'books.csv')
train = pd.read_csv(data_path + 'train_ratings.csv')
test = pd.read_csv(data_path + 'test_ratings.csv')

In [57]:
print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('ratings shape: ', train.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
ratings shape:  (306795, 3)


In [62]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) # 'na' 는 결측치로 인식 X -> 결측치로 변환
users = users.replace('', np.nan) # , , , 으로 입력된 경우 -> 결측치로 변환

In [63]:
users.head()

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins,ontario,canada",NaN,timmins,ontario,canada
1,11400,"ottawa,ontario,canada",49.0,ottawa,ontario,canada
2,11676,"na,na,na",NaN,NaN,NaN,NaN
3,67544,"toronto,ontario,canada",30.0,toronto,ontario,canada
4,85526,"victoria,britishcolumbia,canada",36.0,victoria,britishcolumbia,canada


In [64]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state       3254
location_country     2124
dtype: int64

In [65]:
users[users['location_country'].isna()]

,user_id,location,age,location_city,location_state,location_country
2,11676,"na,na,na",NaN,NaN,NaN,NaN
6,116866,"ottawa,,",NaN,ottawa,NaN,NaN
32,115097,"seattle,,",27.0,seattle,NaN,NaN
49,245827,"albuquerque,,",NaN,albuquerque,NaN,NaN
72,226745,"humble,,",38.0,humble,NaN,NaN
...,...,...,...,...,...,...
67797,257311,"lisbon,maine,",36.0,lisbon,maine,NaN
67929,267240,"houston,,",NaN,houston,NaN,NaN
67930,267276,"sammamish,,",NaN,sammamish,NaN,NaN
68058,276221,"calgary,,",NaN,calgary,NaN,NaN


city 가 존재하는데 state, country 가 결측인 경우 보간


(하지만 오류가 존재한다, city 가 같아도 country 는 다른 경우가 존재 ex:victoria)

In [67]:
# modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
# location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

# location_list = []
# for location in tqdm(modify_location):
#     try:
#         right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
#         location_list.append(right_location)
#     except:
#         pass

# for location in tqdm(location_list):
#     users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
#     users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

# users.isna().sum()

location 이 4개짜리도 존재

In [87]:
def len4_loc(list_):
    i = 0
    if len(list_)!=3:
        i+=1
        print(list_)
    return i

strange_idx = users["location"].str.split(',').apply(lambda x: len4_loc(x))
strange_idx = idx[idx==1].index

['twinfalls', 'idaho', 'idaho', 'usa']
['coalville', 'leicestershire', 'england', 'unitedkingdom']
['tharston', 'norwich', 'england', 'unitedkingdom']
['wichita', '', 'kansas', 'usa']
['paris', 'france', 'iledefrance', 'france']
['chicago', 'il', 'illinois', 'usa']
['ramsey', 'huntingdon', 'cambridgeshire', 'unitedkingdom']
['statesville', '', 'northcarolina', 'usa']
['barboursville', '', 'virginia', 'usa']
['silsden', 'keighley', 'england', 'unitedkingdom']
['aladinma', 'imostate', 'na', 'nigeria']
['fpo', 'ap', 'okinawa', 'japan']
['akron', '', 'ohio', 'usa']
['preston', 'lancashire', 'england', 'unitedkingdom']
['4729pepperwoodave', 'longbeach', 'ca', 'usa']
['merrick', 'acworth', 'newyork', 'georgia', 'usa']
['lakeland', '', 'florida', 'usa']
['washington', 'dc', 'dc', 'usa']
['hauppauge', 'longisland', 'newyork', 'usa']
['vic', 'barcelona', 'catalunya', 'spain']
['hoodsport', 'washington', 'washington', 'usa']
['rochesterhills', 'michigan', 'michigan', 'usa']
['lagrangepark', 'il'

In [86]:
users.iloc[strange_idx]

,user_id,location,age,location_city,location_state,location_country
435,259223,"twinfalls,idaho,idaho,usa",82.0,twinfalls,idaho,idaho
762,14387,"coalville,leicestershire,england,unitedkingdom",35.0,coalville,leicestershire,england
940,38464,"tharston,norwich,england,unitedkingdom",33.0,tharston,norwich,england
1026,51856,"wichita,,kansas,usa",59.0,wichita,NaN,kansas
1044,53688,"paris,france,iledefrance,france",28.0,paris,france,iledefrance
...,...,...,...,...,...,...
66770,202347,"sebabeach,ab,alberta,canada",NaN,sebabeach,ab,alberta
67090,217416,"crivitz,,wisconsin,usa",NaN,crivitz,NaN,wisconsin
67314,229756,"prineville,,oregon,usa",66.0,prineville,NaN,oregon
67584,278215,"nelson,motueka,nz,newzealand",28.0,nelson,motueka,nz


In [38]:
location_users = users[["location_city", "location_state", "location_country"]]

In [20]:
city_na = users[users['location_city'].isnull() & ]
state_na = users[(users['location_city'].notnull()) & (users['location_state'].isnull())]
country_na = users[(users['location_city'].notnull()) & (users['location_country'].isnull())]

country_na[country_na['location_state'].notnull()]

,user_id,location,age,location_city,location_state,location_country
143,57632,"pearland,texas,",NaN,pearland,texas,NaN
273,141444,"westspringfield,massachusetts,",NaN,westspringfield,massachusetts,NaN
651,141518,"vigo,pontevedra,",27.0,vigo,pontevedra,NaN
1358,98026,"yarmouth,novascotia,",40.0,yarmouth,novascotia,NaN
1555,129929,"manchester,newhampshire,",NaN,manchester,newhampshire,NaN
...,...,...,...,...,...,...
67515,240438,"wilfleinsdorf,niederoesterreich,",16.0,wilfleinsdorf,niederoesterreich,NaN
67539,241691,"buenosaires,buenosaires,",24.0,buenosaires,buenosaires,NaN
67619,247213,"stuttgart,badenwrttemberg,",NaN,stuttgart,badenwrttemberg,NaN
67797,257311,"lisbon,maine,",36.0,lisbon,maine,NaN


In [ ]:
users[users['location_city']=='47559kranenburg']